In [1]:

import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing

print( tf.VERSION )


1.15.0


In [13]:
import requests, zipfile, io

r = requests.get( 'https://github.com/asmitstha/data/blob/master/data.zip?raw=true' ) 
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()
print(z)

<zipfile.ZipFile file=<_io.BytesIO object at 0x7f48d1f7b830> mode='r'>


In [16]:
dir_path = r'C:/Users/asmit/Downloads/Chatbot/fyp/chat/data'
files_list = os.listdir(dir_path + os.sep)

questions = list()
answers = list()

for filepath in files_list:
    stream = open( dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(stream)
    conversations = docs['conversations']
    for con in conversations:
        if len( con ) > 2 :
            questions.append(con[0])
            replies = con[ 1 : ]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append( ans )
        elif len( con )> 1:
            questions.append(con[0])
            answers.append(con[1])

answers_with_tags = list()
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )

answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )




tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

FileNotFoundError: ignored

In [0]:
len(answers)


564

In [0]:
from gensim.models import Word2Vec
import re
import string
vocab = []
for word in tokenizer.word_index:
    vocab.append( word )

def tokenize( sentences ):
    tokens_list = []
    vocabulary = []
    for sentence in sentences:
        sentence = str(sentence).lower()
        sentence = re.sub( '[^a-zA-Z]', ' ', sentence )
        result = sentence.translate(str.maketrans("","", string.punctuation))
        tokens = result.split()
        vocabulary += tokens
        tokens_list.append( tokens )
    return tokens_list , vocabulary

p = tokenize( questions + answers )
len(p)







2

In [0]:
# model = Word2Vec( p[ 0 ], min_count=0 ) 
# print(model)

# embedding_matrix = np.zeros( ( VOCAB_SIZE , 100 ) )

In [0]:
# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions ) #Transform each question in a sequence of integers.
maxlen_questions = max( [ len(x) for x in tokenized_questions ] ) # question with maximum length i.e is 50
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' ) #ensure that all sequences in a list have the same length
encoder_input_data = np.array( padded_questions ) #create array
print( encoder_input_data.shape , maxlen_questions )


(564, 22) 22


In [0]:

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers ) #Transform each text in texts in a sequence of integers.
maxlen_answers = max( [ len(x) for x in tokenized_answers ] ) # answer with maximum length i.e is 74
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )  #ensure that all sequences in a list have the same length
decoder_input_data = np.array( padded_answers ) #create array
print( decoder_input_data.shape , maxlen_answers )



(564, 74) 74


In [0]:
# decoder_output_data: Tokenize the answers. Remove the first element from all the tokenized_answers. 
#This is the <START> element which we added earlier.
tokenized_answers = tokenizer.texts_to_sequences( answers ) #Transform each text in texts in a sequence of integers.
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:] #remove first element form all the tokenized_answers.
    
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )  #ensure that all sequences in a list have the same length
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE ) #Converts a class vector (integers) to binary class matrix.
decoder_output_data = np.array( onehot_answers ) #create array of onehot_answers

print( decoder_output_data.shape )


(564, 74, 1894)


In [0]:
encoder_inputs = tf.keras.layers.Input(shape=( None , )) #Input() is used to instantiate a Keras tensor
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs) #Turns positive integers (indexes) into dense vectors of fixed size
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 200)    378800      input_1[0][0]                    

In [0]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=300 ) 

model.save("last_model.h5")

Train on 564 samples
Epoch 1/300
564/564 [==============================] - 13s 22ms/sample - loss: 1.3043
Epoch 2/300
564/564 [==============================] - 2s 4ms/sample - loss: 1.1152
Epoch 3/300
564/564 [==============================] - 2s 4ms/sample - loss: 1.0923
Epoch 4/300
564/564 [==============================] - 2s 4ms/sample - loss: 1.0740
Epoch 5/300
564/564 [==============================] - 2s 4ms/sample - loss: 1.0585
Epoch 6/300
564/564 [==============================] - 2s 4ms/sample - loss: 1.0419
Epoch 7/300
564/564 [==============================] - 2s 4ms/sample - loss: 1.0274
Epoch 8/300
564/564 [==============================] - 2s 4ms/sample - loss: 1.0137
Epoch 9/300
564/564 [==============================] - 2s 4ms/sample - loss: 0.9989
Epoch 10/300
564/564 [==============================] - 2s 4ms/sample - loss: 0.9837
Epoch 11/300
564/564 [==============================] - 3s 5ms/sample - loss: 0.9680
Epoch 12/300
564/564 [=============================

KeyboardInterrupt: ignored

In [0]:
# model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output)
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
# model.load_weights('new_model.h5')


In [0]:
# model.get_weights()

In [0]:
# model.save("new_model.h5")


In [0]:
# from tensorflow.keras.models import load_model
# new_model=load_model('new_model.h5')
# enc_new_model=load_model('encoder_model.h5')
# dec_new_model=load_model('decoder_model.h5')

In [0]:
# import tensorflow as tf 
# new_model = tf.keras.models.load_model('new_model.h5')

In [0]:
# new_model.summary()

In [0]:
# new_model.get_weights()

In [0]:

encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
   
decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding , initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = tf.keras.models.Model( [decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)


In [0]:
# enc_new_model = tf.keras.models.load_model('encoder_model.h5')
# enc_new_model.summary()

In [0]:
# dec_new_model = tf.keras.models.load_model('decoder_model.h5')
# dec_new_model.summary()

In [0]:
# enc_model


In [0]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    words = sentence.translate(str.maketrans("","", string.punctuation)).split()
    print(words)
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')


In [0]:
str_to_tokens("how")

In [0]:


for _ in range(10):
    states_values = encoder_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation ='BOT : '+ ''
    while not stop_condition :
        dec_outputs , h , c = decoder_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )